## Installing Dependecies

In [ ]:
!pip install -r requirements.txt

## loading pretrained model

In [2]:
#loading checkpoint
!gdown 1-HJudsuzTOlVR0x5Mtcnhc0Xgf3Oq1M7

Downloading...
From (original): https://drive.google.com/uc?id=1-HJudsuzTOlVR0x5Mtcnhc0Xgf3Oq1M7
From (redirected): https://drive.google.com/uc?id=1-HJudsuzTOlVR0x5Mtcnhc0Xgf3Oq1M7&confirm=t&uuid=ede3f7d8-d94d-46e2-afb0-ee44a4b1880e
To: /content/score:0.9284939789227805.pth
100% 172M/172M [00:01<00:00, 165MB/s]


## loading dataset

In [3]:
# loading custom ptb-dataset with 17 classes
!gdown 1Rt0I7Svrx77tFMCsNubEQ-cDY8hD-iCk
!gdown 1GWyzUaz_mOwYDbLuopjroIcfngjSJWkD
!unzip r_peaks.zip

In [1]:
from catboost_worker import CatBoostModel, create_conf_interval_gbm
from ecgnet_worker import ECGNetWorker, create_conf_interval_ecgnet
from utils import *
import pandas as pd

# creating ecgworker

In [2]:
ecg_worker = ECGNetWorker(device="cpu")

In [3]:
labels = pd.read_csv("/content/train_val_labels.csv")
ecg_worker.evaluate_data_for_ovr(labels, target_class=12)

evaluate done


In [4]:
ecg_worker.create_torch_dataset(ecg_path="/content/r_peaks/signals/", splits=[0.8, 0.15, 0.05])

train set len: 8320, val set len: 1560


In [20]:
"""
params:

- embedding_size: len of embedding(hidden dim)
- dropout: (float is p, False is not to use it)
- num_layers: (num hidden layers)

"""

params = {
    "embedding_size": 370,
    "dropout" : False,
    "num_layers" : 3
}

ecg_worker.create_model(params)

model succesfully build


In [ ]:
ecg_worker.train_model(n_epochs=1)

  0%|          | 0/130 [00:00<?, ?it/s]

In [ ]:
ecg_worker.inference_model()

## loading ecgworker from checkpoint

In [5]:
ecg_worker.load_model("/content/score:0.9284939789227805.pth", params={"embedding_size" : 370, "dropout" : False, "num_layers" : 3})

## creating confidence interval

In [6]:
mean, bounds = create_conf_interval_ecgnet(ecg_worker, "f1_score", 1000)
print(f"mean score: {mean}, lower bound: {bounds[0]}, upper bound: {bounds[1]}")

100%|██████████| 1000/1000 [00:00<00:00, 26786.80it/s]

mean score: 0.9675626204238921, lower bound: 0.9518304431599229, upper bound: 0.9807803468208093


# Creating dataset for catboost fitting

In [7]:
df = make_dataset(ecg_worker.model, ecg_worker.labels, root_path="/content/r_peaks/signals/", device="cpu", len_embeds=370, len_features=264, save_path="/content")

getting embeds


100%|██████████| 41/41 [08:59<00:00, 13.15s/it]


making dataset


100%|██████████| 10399/10399 [05:35<00:00, 30.97it/s]


In [8]:
df.head(10)

,target,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,e_360,e_361,e_362,e_363,e_364,e_365,e_366,e_367,e_368,e_369
1,1.0,0.007130,0.085201,0.007223,1.727795,5.618940,0.038274,0.049312,0.002008,0.000829,...,0.0,0.0,1.484588,0.0,0.0,0.0,0.0,0.373570,0.0,0.0
2,1.0,0.000264,0.075248,0.005674,1.766583,5.816614,0.033901,0.042595,0.001638,0.000634,...,0.0,0.0,1.578388,0.0,0.0,0.0,0.0,0.342532,0.0,0.0
3,1.0,0.000906,0.082294,0.006785,1.743166,5.886365,0.038649,0.047210,0.002176,0.000925,...,0.0,0.0,1.400864,0.0,0.0,0.0,0.0,0.354679,0.0,0.0
4,1.0,-0.000355,0.076745,0.005901,1.778336,5.792177,0.034315,0.043155,0.001734,0.000686,...,0.0,0.0,1.687698,0.0,0.0,0.0,0.0,0.311500,0.0,0.0
5,1.0,-0.000844,0.074410,0.005547,1.882930,5.898902,0.030272,0.039518,0.001680,0.000655,...,0.0,0.0,1.639354,0.0,0.0,0.0,0.0,0.266376,0.0,0.0
6,1.0,0.004144,0.088000,0.007742,1.774749,5.684229,0.039525,0.049584,0.002508,0.001104,...,0.0,0.0,1.479142,0.0,0.0,0.0,0.0,0.340393,0.0,0.0
7,1.0,-0.005503,0.081358,0.006602,1.700600,5.636077,0.037969,0.047841,0.001917,0.000805,...,0.0,0.0,1.505485,0.0,0.0,0.0,0.0,0.314118,0.0,0.0
8,1.0,0.003364,0.078106,0.006101,2.123967,5.819364,0.024548,0.036774,0.001926,0.000772,...,0.0,0.0,1.498494,0.0,0.0,0.0,0.0,0.334927,0.0,0.0
9,1.0,0.001711,0.085133,0.007259,1.674942,5.417973,0.039807,0.050828,0.001988,0.000832,...,0.0,0.0,1.412737,0.0,0.0,0.0,0.0,0.393913,0.0,0.0
10,1.0,-0.007174,0.075812,0.005707,1.783382,5.594287,0.032320,0.042511,0.001623,0.000633,...,0.0,0.0,1.491568,0.0,0.0,0.0,0.0,0.254924,0.0,0.0


# fitting CatBoost

In [10]:
cb_df = pd.read_csv("/content/dataset_cbm.csv")

gbm = CatBoostModel(seed=42, device="cpu")

In [11]:
gbm.init_data(cb_df, target_column="target", splits=[0.75, 0.1, 0.15], embedding_features=[])

/usr/local/lib/python3.10/dist-packages/pandas/core/series.py:917: RuntimeWarning: invalid value encountered in cast
  arr = np.asarray(values, dtype=dtype)


In [ ]:
gbm.tune_params(n_trials=20, timeout_study=3600)
gbm.get_best_params_optuna()

[I 2024-04-08 12:11:12,086] A new study created in memory with name: no-name-0c053556-e4ac-4133-b2a7-6246582d7c94


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2024-04-08 12:11:15,852] Trial 0 finished with value: 0.967992240543162 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.07858536406599222, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.1408686626948248}. Best is trial 0 with value: 0.967992240543162.
[I 2024-04-08 12:11:26,953] Trial 1 finished with value: 0.97265625 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.030118379260155828, 'depth': 6, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.5561660458905046}. Best is trial 1 with value: 0.97265625.
[I 2024-04-08 12:12:42,281] Trial 2 finished with value: 0.975609756097561 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.042607983947643875, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.9752328070068176}. Best is trial 2 with value: 0.975609756097561.
[I 2024-04-08 12:16:27,918] Trial 3 finished with value: 0.9718172983479106 and p

In [ ]:
gbm.best_optuna_model.save_model("/content/optuna50_best.cbm")

In [ ]:
gbm.best_trial.params

{'objective': 'Logloss',
 'colsample_bylevel': 0.06862982519841486,
 'depth': 8,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bernoulli',
 'subsample': 0.9885331636414759}

In [ ]:
class_params = gbm.best_trial.params

gbm.init_model(parameters=class_params, task="classification")

In [ ]:
fit_params = {
    "verbose" : 50,
}

gbm.fit_model(parameters=fit_params)

Learning rate set to 0.052633
0:	learn: 0.5805515	test: 0.5779505	best: 0.5779505 (0)	total: 224ms	remaining: 3m 43s
50:	learn: 0.0363867	test: 0.0304287	best: 0.0304287 (50)	total: 11.8s	remaining: 3m 40s
100:	learn: 0.0226356	test: 0.0258995	best: 0.0258995 (100)	total: 22.6s	remaining: 3m 21s
150:	learn: 0.0155648	test: 0.0238599	best: 0.0238267 (148)	total: 33.3s	remaining: 3m 6s
200:	learn: 0.0111168	test: 0.0227637	best: 0.0226911 (199)	total: 44.7s	remaining: 2m 57s
250:	learn: 0.0087817	test: 0.0221533	best: 0.0221533 (250)	total: 56.1s	remaining: 2m 47s
300:	learn: 0.0072226	test: 0.0216010	best: 0.0216010 (300)	total: 1m 8s	remaining: 2m 39s
350:	learn: 0.0064037	test: 0.0212882	best: 0.0212790 (347)	total: 1m 18s	remaining: 2m 25s
400:	learn: 0.0057981	test: 0.0209434	best: 0.0209327 (397)	total: 1m 30s	remaining: 2m 14s
450:	learn: 0.0053008	test: 0.0208651	best: 0.0208229 (436)	total: 1m 41s	remaining: 2m 3s
500:	learn: 0.0047809	test: 0.0208561	best: 0.0208229 (436)	total

In [ ]:
gbm.inference_model(task_type="CPU")

{'f1_score': 0.9766536964980543, 'accuracy_score': 0.9846153846153847}

## creating confidence interval

In [ ]:
mean, bounds = create_conf_interval_gbm(gbm, "f1_score", 1000)
print()
print(f"mean score: {mean}, lower bound: {bounds[0]}, upper bound: {bounds[1]}")